In [25]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import mean, to_date, sum, col, year, month, avg, count

spark = SparkSession.builder.appName("Books and authors").getOrCreate()

df1 = spark.read.csv("/home/jovyan/work/code/PySpark_test/config/authors.csv", header=True, inferSchema=True)
df2 = spark.read.csv("/home/jovyan/work/code/PySpark_test/config/books.csv", header=True, inferSchema=True)

df1 = df1.withColumn("birth_date", to_date(df1["birth_date"], "yyyy-MM-dd"))
df2= df2.withColumn("publish_date", to_date(df2["publish_date"], "yyyy-MM-dd"))

df_join = df2.join(df1, "author_id", "inner")

top5_authors = df_join.groupBy("author_id", "name").agg(sum("price").alias("total_revenue")) \
.orderBy(col("total_revenue").desc()).limit(5)

genre_count = df_join.groupBy("genre").agg(count("title").alias("count")) \
.orderBy(col("count").desc())

avg_price = df_join.groupBy("author_id", "name").agg(avg("price").alias("average_price")) \
.orderBy(col("average_price").desc())

df_gone2000 = df_join.filter(year("publish_date") >= 2000)

books_gone2000 = df_gone2000.orderBy(col("price").desc())
# df1.printSchema()
# df2.printSchema()
# top5_authors.show()
# genre_count.show()
# avg_price.show()
books_gone2000.show()
df_join.show()
# df1.show()
# df2.show()

+---------+-------+-------+-----------+-----+------------+---------+----------+---------+
|author_id|book_id|  title|      genre|price|publish_date|     name|birth_date|  country|
+---------+-------+-------+-----------+-----+------------+---------+----------+---------+
|        7|     20|Book_20|    Mystery|91.48|  2018-12-31| Author_7|1990-12-31|      USA|
|        5|     19|Book_19|    Science|88.83|  2016-12-31| Author_5|1980-12-31|      USA|
|        8|     15|Book_15|    Fantasy| 60.0|  2008-12-31| Author_8|1995-12-31|Australia|
|        6|     17|Book_17|    Fantasy|47.57|  2012-12-31| Author_6|1985-12-31|      USA|
|        1|     18|Book_18|Non-Fiction|43.92|  2014-12-31| Author_1|1960-12-31|    India|
|        2|     16|Book_16|    Fiction|36.22|  2010-12-31| Author_2|1965-12-31|   Canada|
|       10|     11|Book_11|Non-Fiction| 31.7|  2000-12-31|Author_10|2005-12-31|    India|
|        8|     12|Book_12|Non-Fiction|31.02|  2002-12-31| Author_8|1995-12-31|Australia|
|        1